In [1]:
import whisperx
import gc
import os
from dotenv import load_dotenv

load_dotenv()

token = os.environ.get("HUGGINGFACE_ACCESS_TOKEN")

device = "cuda"
audio_file = "544_short.mp3"
batch_size = 16
compute_type = "float16"

model = whisperx.load_model("large-v2", device, compute_type=compute_type)

# save model to local path (optional)
# model_dir = "/path/"
# model = whisperx.load_model("large-v2", device, compute_type=compute_type, download_root=model_dir)

audio = whisperx.load_audio(audio_file)

Lightning automatically upgraded your loaded checkpoint from v1.5.4 to v2.3.3. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../../.cache/torch/whisperx-vad-segmentation.bin`


No language specified, language will be first be detected for each audio file (increases inference time).
Model was trained with pyannote.audio 0.0.1, yours is 3.3.1. Bad things might happen unless you revert pyannote.audio to 0.x.
Model was trained with torch 1.10.0+cu102, yours is 2.3.1+cu121. Bad things might happen unless you revert torch to 1.x.


In [5]:
result = model.transcribe(audio, batch_size=batch_size)
# print(result["segments"])

model_a, metadata = whisperx.load_align_model(
    language_code=result["language"], device=device
)
result = whisperx.align(
    result["segments"], model_a, metadata, audio, device, return_char_alignments=False
)

# print(result["segments"])

diarize_model = whisperx.DiarizationPipeline(use_auth_token=token, device=device)

diarize_segments = diarize_model(audio)
diarize_model(audio, min_speakers=2, max_speakers=2)

result = whisperx.assign_word_speakers(diarize_segments, result)

print(diarize_segments)

Detected language: en (1.00) in first 30s of audio...
                               segment label     speaker       start  \
0    [ 00:00:00.030 -->  00:00:02.427]     A  SPEAKER_00    0.030969   
1    [ 00:00:02.612 -->  00:00:06.915]     B  SPEAKER_01    2.612844   
2    [ 00:00:07.354 -->  00:00:07.371]     C  SPEAKER_01    7.354719   
3    [ 00:00:07.371 -->  00:00:17.192]     D  SPEAKER_00    7.371594   
4    [ 00:00:17.631 -->  00:00:19.690]     E  SPEAKER_00   17.631594   
..                                 ...   ...         ...         ...   
131  [ 00:09:04.755 -->  00:09:16.382]    EB  SPEAKER_00  544.755969   
132  [ 00:09:16.180 -->  00:09:27.284]    EC  SPEAKER_01  556.180344   
133  [ 00:09:26.035 -->  00:09:59.042]    ED  SPEAKER_00  566.035344   
134  [ 00:09:33.173 -->  00:09:34.034]    EE  SPEAKER_01  573.173469   
135  [ 00:09:36.413 -->  00:09:36.936]    EF  SPEAKER_01  576.413469   

            end  intersection       union  
0      2.427219   -593.409781  596.38

In [4]:
lines = result["segments"]
for line in lines:
    start, end, text, _, speaker = line.values()
    print(speaker, text)

SPEAKER_00  Well, I have a little bit of a scratchy voice.
SPEAKER_01 Yeah, I can tell it because when we start you say Berlin.
SPEAKER_00  Berlin, hi Berlin.
SPEAKER_00 Hi Berlin, hello Berlin.
SPEAKER_00 You know, this is the kind of scratchy voice that, boy, Dan Benjamin would cancel a show in a heartbeat.
SPEAKER_00 Okay.
SPEAKER_00 With a scratchy voice like this one.
SPEAKER_01 Well, I guess we're fortunate we can, we'll edit around him for this one.
SPEAKER_00 Okay, okay, good.
SPEAKER_00 But I don't know what, last night, I was like, hmm, my voice is a little scratchy.
SPEAKER_00 And then I thought, oh no, no, no, no, I can't be getting sick, I hate getting sick.
SPEAKER_01  You are if I may say and listen, I've known you for at least I don't know five eight years now You you are not a good sick person You are you are a pillar of the community in so many ways for so many things You're always putting it on the line for everybody.
SPEAKER_01 But when you get sick, you become like